In [ ]:
# Setup
import nltk 
nltk.download('framenet_v17')

In [ ]:
# Get the template
from typing import Dict
from app.src.helpers.template_getter import get_template
contract = get_template('prop_t')

parameter_dict: Dict[str,str] = {}

contract.print_all_strings()

In [ ]:
print(contract.to_sym())

In [ ]:
from app.src.user_scripts.main_ops.user_deps import get_dependencies
from app.src.user_scripts.main_ops.refine_parm import refine_parm
from app.src.user_scripts.main_ops.add_term import add_term
from app.src.user_scripts.main_ops.add_dm import add_dm

deps = get_dependencies()

In [ ]:
# User chooses the operation
ops_dict = {
    1: 'Customize a Parameter',
    2: 'Add a termination condition',
    3: 'Add a new domain event'
}

for opi in ops_dict:
    print(f'{opi} - {ops_dict[opi]}')
    
selected_op = int(input('Select an operation: '))

print(f'\nSelected operation: {ops_dict[selected_op]}\n')

In [ ]:
# Select the parameter
## TODO: Need a func to get the right parms (maybe include it with the contract collection)
#from app.templates.sample.t.nl_template import sample_parameters
from app.templates.prop_mgmt.t.nl_template import parameters

if selected_op == 1:
    refine_parm(deps, contract, parameters)

elif selected_op == 2:
    add_term(deps, contract)

elif selected_op == 3: 
    add_dm(deps, contract)

In [ ]:
print(contract.to_sym())

## Step through

In [ ]:
from app.src.operations.helpers.default_event_getter import DefaultEventGetter

# Extract the default event
deg = DefaultEventGetter()

# Handle the case where there is none...
default_event = deg.get(contract.contract_spec, parm_config)

if default_event:
    print(f'Default Event: {default_event.to_sym()}')
else:
    print('No default event')

In [ ]:
#default_event = None

In [ ]:
from app.src.frames.frame_checker_constuctor import FrameCheckerConstructor

frame_checker = FrameCheckerConstructor.construct()
frames = frame_checker.check_all_frames(selection.nodes)

for x in frames:
    print(type(x), x.to_text())

main_frame = frames[0]
op_code = main_frame.op_code

In [ ]:
# Create the basket
from app.classes.grammar.selected_node import Basket
basket = Basket()
basket.default_event = default_event
basket.initial_norm = contract.contract_spec.__dict__[parm_config.norm_type][parm_config.norm_id]

In [ ]:
# Generate the Symboleo
symboleo_obj = selection.to_obj(basket)
print(symboleo_obj.to_sym())

In [ ]:
# Updating
from app.src.operations.parameter_updater_builder import ParameterUpdaterBuilder

parm_updater = ParameterUpdaterBuilder.build()

updated_contract = parm_updater.update(op_code, contract, symboleo_obj, parm_config)


In [ ]:
print(updated_contract.to_sym())